# Get and clean the mobility network data

## OpenSwissData road network:

In [ ]:
import geopandas as gpd

roads = gpd.read_file("../data/road_networks/swisstlm3d_roads_2025.gpkg")

#### Crop the roads to Vaud:

In [ ]:
import osmnx as ox
from geopandas import overlay

place = "Vaud, Switzerland"
bound = ox.geocode_to_gdf(place)

target_crs = "EPSG:2056" # roads_gdf crs
bound = bound.to_crs(target_crs)

roads = overlay(roads, bound, how='intersection')

In [ ]:
print(roads.columns)

In [ ]:
print("The types of roads available in this dataset are:")
print(roads["object_type"].unique())

print("\n There are also hierarchy levels for the roads:")
print(roads["hierarchy_level"].unique())

#### Filter for main roads:
from the list of type of roads, selct the ones to keep in teh network.

In [ ]:
main_roads = [
    'Autobahn',
    # 'Autostrasse',
    # 'Ausfahrt',
    # 'Einfahrt',
    # 'Zufahrt'
]
roads_clean = roads[roads["object_type"].isin(main_roads)].copy()


In [ ]:
print(roads_clean["display_name"].unique())

In [ ]:
print(roads_clean["hierarchy_level"].unique())

## OSMNX road network:


In [ ]:
place = "Vaud, Switzerland"
G = ox.graph_from_place(place, network_type="drive")
roads_ox = ox.graph_to_gdfs(G, nodes=False, edges=True).reset_index()
roads_ox = roads_ox.to_crs(epsg=2056)

In [ ]:
print(roads_ox.columns)

In [ ]:
import itertools
unique_highways = set(itertools.chain.from_iterable(roads_ox["highway"]))
print(unique_highways)

In [ ]:
roads_ox_clean = roads_ox[roads_ox["highway"].isin(
    ["motorway"]
)].copy()

In [ ]:
print(roads_ox_clean["ref"].unique())
a9 = roads_ox[(roads_ox["ref"] == "A9")|(roads_ox["ref"] == "A1;A9")].copy()
print(a9.head())

#### Visualize the selected roads:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(30, 30))
# roads_clean.plot(ax=ax, color='blue')
# roads_ox_clean.plot(ax=ax, color='blue')
a9.plot(ax=ax, color='blue')
ax.axis('off')


#### Save the cleaned roads to a GeoPackage file for further use in the buffers generation notebook:

In [ ]:
# roads_clean.to_file("road_networks/vaud_main_roads.gpkg", driver="GPKG")

## Rail network data

In [ ]:
import osmnx as ox

place = "Vaud, Switzerland"
rail_tags = {'railway': ['rail']} ## + rail?
subway_tags = {'railway': [ 'subway', 'tram']}
railway = ox.features.features_from_place(place, tags=rail_tags)
# subway = ox.features.features_from_place(place, tags=subway_tags)

print(railway.columns)
# print(subway.columns)

In [ ]:
print(railway['ref'].unique())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Get unique refs, ignoring NaN for coloring
unique_refs = railway['ref'].dropna().unique()
n_refs = len(unique_refs)
colors = cm.get_cmap('tab20', n_refs)  # or 'tab10', 'hsv', etc.

fig, ax = plt.subplots(figsize=(12, 8))

for i, ref in enumerate(unique_refs):
    # if ref == '200' or ref == '100' or ref == '150':
        sub = railway[railway['ref'] == ref]
        sub.plot(ax=ax, color=colors(i), label=str(ref), linewidth=3)

plt.legend(title="ref", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title(f"Railway lines in {place} by 'ref'")
plt.tight_layout()
plt.show()

We will select only the railway lines with ref '100' and '150' for further analysis, as these are lines parallel to the A9 motorway:
- in Lausanne : 100, 150
- in Vaud : 100, 150, 200

In [ ]:
railway = railway[railway['ref'].isin(['100', '150'])].copy()
railway = railway.to_crs(epsg=2056)  # Convert to Swiss coordinate system
railway.to_file(f"../data/railway_networks/Vaud_rail_100_150_200.gpkg", driver="GPKG")

In [ ]:
print(railway['maxspeed'].unique())

In [ ]:
print(railway['tracks'].unique())

In [ ]:
print(railway.crs)

## Correct the habitat data

In [ ]:
import pandas as pd
habitat_distr = pd.read_csv("../data/habitats/habitats_in_A9_buffers.csv")

# Correct '4.0.0.0.3' to '4.0.3' in TypoCH column
habitat_distr['TypoCH'] = habitat_distr['TypoCH'].replace('4.0.0.0.3', '4.0.3')

# Save the corrected DataFrame back to a CSV file
habitat_distr.to_csv("../data/habitats/habitats_in_A9_buffers.csv", index=False)

In [ ]:
typoch_names = pd.read_csv("../data/habitats/typoCH_names.csv")

def normalize_typo(typo):
    # Remove any '-'
    # Remove any stray spaces
    typo = str(typo).replace('-','')
    typo = typo.strip()
    return typo

typoch_names['normalized_typo'] = typoch_names['Typo'].apply(normalize_typo)

# Save the updated typoCH names
typoch_names.to_csv("../data/habitats/typoCH_names.csv", index=False)

In [ ]:
import pandas as pd
habitats_rail = pd.read_csv("../data/habitats/habitats_in_vaud_railway_buffers.csv")

In [ ]:
print(habitats_rail.head())

In [ ]:
print(habitats_rail['habitat_area'].unique())

In [ ]:
print(min(habitats_rail['buffer_id'].unique()))